In [1]:
import os
import copy
import torch
import random
import torchvision
import numpy as np
import torch.nn as nn
from torch.nn import Parameter
from scipy import interpolate
import matplotlib.pyplot as plt

In [20]:
def dataupsample_2d(x,axis,num_up,method = 'nearest'):
  #x = np.transpose(x)
    if axis == 0:
        x1 = np.linspace(0,x.shape[0]-1,x.shape[0])
        x_new = np.linspace(0,x.shape[0]-1,num_up)
        gxu = np.zeros((num_up,x.shape[1]),dtype=np.single)
        for i in range(x.shape[1]):
            f = interpolate.interp1d(x1,x[:,i],kind = method)
            gxu[:,i] = f(x_new)
    elif axis == 1:
        x1 = np.linspace(0,x.shape[1]-1,x.shape[1])
        x_new = np.linspace(0,x.shape[1]-1,num_up)
        gxu = np.zeros((x.shape[0],num_up),dtype=np.single)
        for i in range(x.shape[0]):
            f = interpolate.interp1d(x1,x[i,:],kind = method)
            gxu[i,:] = f(x_new)
    return gxu

def data_augment(fx):
    method = random.sample([0,1],1)[0]
    if method==0:
        crop_width = int(128*round(random.uniform(0.7,0.9),2))
        x1 = int(random.uniform(0,128-crop_width))
        fx = fx[:,x1:x1+crop_width]
        fx = dataupsample_2d(x=fx,axis=1,num_up=128,method='nearest')
    elif method==1:
        crop_width = int(128*round(random.uniform(0.7,0.9),2))
        x1 = int(random.uniform(0,128-crop_width))
        fx = fx[:,x1:x1+crop_width]
        fx = dataupsample_2d(x=fx,axis=1,num_up=128,method='nearest')
        fx = np.flip(fx,1)
    return fx    

In [21]:
classes = ["parallel","clinoform","fill","hummocky","chaotic"]
nums = [2000,1500,1500,1500,1500]

for ip in range(5):
    ci = classes[ip]
    ni = nums[ip]
    for nii in range(ni):
        read_dir = "../datasets/"+ci+"_classification/"+str(nii)+".dat"
        save_dir = "./datasets/"+ci+"_classification/"
        fx = np.fromfile(read_dir,dtype=np.int32).reshape(128,128).T
        np.save(save_dir + str(nii) + ".npy",fx)
        fx1 = data_augment(fx).astype(np.int32)
        np.save(save_dir + str(nums[ip]+nii) + ".npy",fx1)